In [5]:
import numpy as np
import sys
import os
import pickle
import glob
import plotly.graph_objects as go
from plotly.subplots import make_subplots

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from DEFAULTS import PARENT_PATH
import sim.ap_field_ar as ap
import sim.far_field_mp as ff
import sim.tele_geo_ar as tg
import sim.pan_mod_ar as pm

In [6]:
# create panel offsets randomly
save = 0
error_rms = 0
adj_1_A = np.random.randn(1092) * error_rms
adj_2_A = np.random.randn(1092) * error_rms
panel_model2 = pm.panel_model_from_adjuster_offsets(
    2, adj_2_A, 1, save
)  # Panel Model on M2
panel_model1 = pm.panel_model_from_adjuster_offsets(
    1, adj_1_A, 1, save
)  # Panel Model on M1

In [7]:
# ###################### define telescope geometry
tele_geo = tg.TelescopeGeometry()
tele_geo.N_scan = 100  # pixels in 1D of grid
# tele_geo.de_ang = 0.2 / 60 * np.pi / 180  # angle increment of telescope scan [rad]
de_ang = 1 / 60 * np.pi / 180  # Far-field angle increment, arcmin = 1/60 degree

tele_geo.k = 2 * np.pi / tele_geo.lambda_  # wavenumber [1/m]
# tele_geo.th_fwhp = 44 * np.pi / 180  # full width half power of source feed [rad]
tele_geo.z_tow = 1e3 #[m]
# Azimuth and Elevation center [rad]
tele_geo.az0 = 0.0
tele_geo.el0 = np.arctan(-tele_geo.y_tow / tele_geo.z_tow)
# el0 = 0.005
# position of the receiver feed [mm]
P_rx = np.array([0, 209.920654, 0])
# P_rx = np.array([0, 0, 0])
[tele_geo.rx_x, tele_geo.rx_y, tele_geo.rx_z] = P_rx/1e3

# arrays of pointing angles of rays
angle_size = 0.28
theta_a, theta_b, theta_N = -np.pi / 2 - angle_size, -np.pi / 2 + angle_size, 100
phi_a, phi_b, phi_N = np.pi / 2 - angle_size, np.pi / 2 + angle_size, 100
theta = np.linspace(theta_a, theta_b, theta_N)
phi = np.linspace(phi_a, phi_b, phi_N)

# get parameters from telescope geometry
tg_th_1, tg_th2, tg_z_ap = tele_geo.th_1, tele_geo.th2, tele_geo.z_ap
tg_th_fwhp, tg_F_2 = tele_geo.th_fwhp, tele_geo.F_2

# ray tracing
rxmirror = ap.ray_mirror_pts(P_rx, tg_th2, tg_F_2, theta, phi)
apout = ap.aperature_fields_from_panel_model(panel_model1, panel_model2, \
                                    P_rx, tg_th_1, tg_th2, tg_z_ap, tg_th_fwhp, \
                                    theta, phi, rxmirror
                                    )
farbeam = ff.far_field_sim(apout, tele_geo, None)

ValueError: operands could not be broadcast together with shapes (10000,) (40000,) 

In [ ]:
path = "E:\\Holography\\lat_holocal\\data\\chesmore\\sim_out\\rx000pys"
filename_base = "skysim_data_nopanelerrors"

az = np.linspace(tele_geo.az0-tele_geo.N_scan*tele_geo.de_ang, tele_geo.az0+tele_geo.N_scan*tele_geo.de_ang, 2*tele_geo.N_scan)
el = np.linspace(tele_geo.el0-tele_geo.N_scan*tele_geo.de_ang, tele_geo.el0+tele_geo.N_scan*tele_geo.de_ang, 2*tele_geo.N_scan)
adj_m1 = adj_1_A
adj_m2 = adj_2_A
beam = np.reshape(farbeam[2], (tele_geo.N_scan*2, tele_geo.N_scan*2))
savedata = dict(
            adj_err_1=adj_m1,
            adj_err_2=adj_m2, 
            field_sky=beam, 
            az=az,
            el=el
            )

with open(f"{path}/{filename_base}.pys", "wb") as file:
    pickle.dump(savedata, file)

################## initialize plots ##################
layout = go.Layout(title='Beam on sky', autosize=True,
            width=1000, height=500, 
            margin=dict(l=50, r=50, b=65, t=90),
            xaxis1 = dict(title="x [mrad]"),
            yaxis1 = dict(scaleanchor = 'x', title="y [mrad]"),
            xaxis2 = dict(scaleanchor = "x", title="x [mrad]"),
            yaxis2 = dict(scaleanchor = "y"),
            )
fig = make_subplots(rows=1, cols=2, shared_yaxes=False, 
                shared_xaxes=True, subplot_titles=["Phase [rad]", "Intensity [dB]"])
fig.update_layout(layout)

# save plots
x = az*1e3
y = el*1e3
field_fp = beam
field_fp = field_fp / np.max(np.abs(field_fp)) # normalize the field
plot_1 = go.Heatmap(
        x=x, 
        y=y,
        z = np.angle(field_fp),
        colorscale='Twilight',
        showscale=True, colorbar=dict(len=0.8, x=0.44),
        showlegend = False,
        # hoverinfo='name', 
        name="phase"
        )
plot_2 = go.Heatmap( 
        x=x, 
        y=y,
        z = 20*np.log10(np.abs(field_fp)),
        # z= np.abs(field_fp*field_fp),
        colorscale='Magma',
        showscale=True, 
        colorbar=dict(len=0.8, x=1), 
        showlegend = False,
        # hoverinfo='name', 
        name="power"
        ) 
fig.data = []
fig.add_trace(plot_1, row=1, col=1)
fig.add_trace(plot_2, row=1, col=2)
fig.write_image(f"{path}/{filename_base}.png")